This script loop infinitely to looking for already scored models to prune from twinews-rankings

In [ ]:
# tipi 86
# killbill modelspruner ; cd ~/twinews-logs ; jupython -o nohup-modelspruner-$HOSTNAME.out --venv st-venv ~/Workspace/Python/Datasets/Twinews/twinews/evaluation/modelspruner.ipynb

In [ ]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
isNotebook = '__file__' not in locals()

In [ ]:
TEST = False # isNotebook, True, False

In [ ]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from databasetools.mongo import *
from newstools.goodarticle.utils import *
from nlptools.preprocessing import *
from nlptools.news import parser as newsParser
from machinelearning.iterator import *
from twinews.utils import *
from twinews.evaluation import metrics
from twinews.evaluation.utils import *
import time
import pymongo
from datetime import datetime

In [ ]:
logger = Logger(tmpDir('logs') + "/modelspruner.log") if isNotebook else Logger("modelspruner-" + getHostname() + ".log")
tt = TicToc(logger=logger)
tt.tic()

In [ ]:
iterations = 1 if TEST else 10000000
sleep = 0 if TEST else 300
exceptionSleep = 10
minLifeTime = 30 if TEST else 6 * 3600

In [ ]:
modelsToPrune = {'combin'}

In [ ]:
# We get scores collection and the rankings GridFS:
twinewsScores = getTwinewsScores(logger=logger)
twinewsRankings = getTwinewsRankings(logger=logger)

In [ ]:
# Init of metrics:
metrics = {'ndcg', 'ndcg@10', 'ndcg@100', 'mrr', 'p@10', 'p@100', 'map',
          'div@100', 'style-div@100', 'topic-div@100',}

In [ ]:
# For a certain amount of iterations:
for iteration in range(iterations):
    try:
        # We get only files that take a lot of disk:
        rows = []
        for model in modelsToPrune:
            rows += list(twinewsRankings.collection.find({'meta.model': model, 'length': {'$gt': 100}}))
        # We check meta.model:
        for row in rows:
            assert row['meta']['model'] in modelsToPrune
        # We keep only ids:
        for i in range(len(rows)):
            rows[i] = rows[i]['id']
        # We keep only old ones according to the upload date in twinews.scores:
        idsToPrune = set()
        for currentId in rows:
            rows = []
            for metric in metrics:
                rows.append(twinewsScores.findOne({'id': currentId, 'metric': metric}))
            if None not in rows:
                row = rows[0]
                _id = row['_id']
                ts = datetime.timestamp(_id.generation_time)
                now = time.time()
                if now - ts >= minLifeTime:
                    idsToPrune.add(currentId)
        # We log infos:
        log("To prune: " + b(idsToPrune) + " (" + str(len(idsToPrune)) + ")", logger)
        # We prune all:
        for currentId in idsToPrune:
            log("Pruning " + currentId, logger)
            if not TEST:
                pruneRankings(currentId)
    except Exception as e:
        if isNotebook:
            raise e
        else:
            logException(e, logger)
            time.sleep(exceptionSleep)
    if sleep > 0:
        log("Sleeping " + str(sleep) + " seconds for the iteration " + str(iteration) + " on " + str(iterations) + "...", logger)
    time.sleep(sleep)